In [1]:
import pandas as pd
import json

In [2]:
def generate_data():
    with open("results.jsonl", "r", encoding="utf-8") as file:
        for line in file.readlines():
            student: dict = json.loads(line[:-1])
            key = list(student.keys())[0]
            idx = student.pop(key)
            student["idx"] = idx
            yield student

In [3]:
students_data = generate_data()
df = pd.DataFrame(students_data)

In [4]:
df.replace(r'\t{2,}', r" ", regex=True, inplace=True)
df.columns = [col.replace(':', '') for col in df.columns]
df['النتيجة'] = df['النتيجة'].str.replace(r'\r', '', regex=True)
df['النتيجة'] = df['النتيجة'].str.replace(r'\n', '', regex=True)
df['النتيجة'] = df['النتيجة'].str.replace(':', '')
df['النتيجة'] = df['النتيجة'].str.replace('النتيجة', '',)

In [5]:
filt1 = df['title'] == 'المجموع'
filt2 = df['title'] == 'التربية الدينية'
filt3 = df['title'] == 'المجموع العام'

df = df.loc[~(filt1 | filt2 | filt3)]

In [6]:
mapper = {
  'title': 'المادة',
  'mark': 'العلامة',
  'min': 'العلامة الدنيا',
  'max': 'العلامة العظمى',
}
df.rename(mapper=mapper, axis='columns', inplace=True)

In [7]:
df['العلامة'] = df['العلامة'].astype(int)

In [17]:
df['النتيجة'] = df['النتيجة'].str.strip()
df['المادة'] = df['المادة'].str.strip()
df.to_excel('data.xlsx')

C:\Users\wpu0001\AppData\Local\Temp\ipykernel_7756\1130389650.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['النتيجة'] = df['النتيجة'].str.strip()
C:\Users\wpu0001\AppData\Local\Temp\ipykernel_7756\1130389650.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['المادة'] = df['المادة'].str.strip()


In [9]:
df = df.loc[df['النتيجة'] != 'راسب']

In [10]:
results = df.pivot_table(
  values= ['العلامة'],
  index=['idx', 'الشهادة', 'العام الدراسي', 'المحافظة', 'الاسم الكامل', 'اسم الأم', 'المدرسة', 'النتيجة'],
  aggfunc='sum',
  fill_value=0
).reset_index()

In [11]:
english = df.loc[df['المادة'] == 'اللغة الإنكليزية'][['idx', 'الاسم الكامل', 'العلامة']]
english.columns = ['idx', 'الاسم الكامل', 'english']
french = df.loc[df['المادة'] == 'اللغة الفرنسية'][['idx', 'الاسم الكامل', 'العلامة']]
french.columns = ['idx', 'الاسم الكامل', 'french']
# russian = df.loc[df['المادة'] == 'اللغة الروسية'][['idx', 'الاسم الكامل', 'العلامة']]
# russian.columns = ['idx', 'الاسم الكامل', 'russian']

In [12]:
results = results.merge(english, how='inner', left_on=['idx' ,'الاسم الكامل'], right_on=['idx' ,'الاسم الكامل'])
results = results.merge(french, how='inner', left_on=['idx' ,'الاسم الكامل'], right_on=['idx' ,'الاسم الكامل'])
# results = results.merge(russian, how='outer', left_on=['idx' ,'الاسم الكامل'], right_on=['idx' ,'الاسم الكامل'])

In [13]:
results['english'] = results['english'].astype(int)
results['french'] = results['french'].astype(int)
# results['russian'] = results['russian'].astype(int)
# results

In [14]:
results['final_mark'] = results.apply(
  lambda x: x['العلامة'] - min(x['english'], x['french']),
  axis='columns'
  )

In [15]:
df['الاسم الكامل'].value_counts()

الاسم الكامل
علي محمد العلي                  48
علي احمد الابراهيم              16
زينه محمد عيسى                  16
علي احمد ابراهيم                16
هبه محمد الحسن                  16
                                ..
سدرة ايمن السعدي                 7
سيدرا راشد محمد                  7
سدره أنس الشيرازي الصباغ         7
ريناس كمال يزبك                  7
عبد الباسط عبد الحميد نشناشي     7
Name: count, Length: 17725, dtype: int64

In [16]:
results['idx'] = results['idx'].astype(int)
results.sort_values(['idx'])

,idx,الشهادة,العام الدراسي,المحافظة,الاسم الكامل,اسم الأم,المدرسة,النتيجة,العلامة,english,french,final_mark
13160,4,الفرع الأدبي,2023-2022,حمص,عبد الباسط عبد الحميد نشناشي,مايا,ثا / رابطة الخريجين الجامعيين الخاصة,ناجح,1979,286,340,1693
13159,4,الفرع الأدبي,2023-2022,حماه,احمد مصطفى المصري,مؤمنه,ثا ابن رشد,ناجح,1444,176,164,1280
13673,5,الفرع الأدبي,2023-2022,حماه,ادهم رضوان الملحم,طرفه,ثا ابن رشد,ناجح,1706,191,182,1524
14472,6,الفرع الأدبي,2023-2022,حماه,اسامه نصر الدين المحمد,منتهى,ثا ابن رشد,ناجح,1937,160,160,1777
15082,7,الفرع الأدبي,2023-2022,حماه,أمير ياسر السراج,رزان,ثا ابن رشد,ناجح,1662,173,257,1489
...,...,...,...,...,...,...,...,...,...,...,...,...
12620,26925,الفرع العلمي,2023-2022,حمص,يارا عبدالله الزير,عائده,دراسة حرة,ناجح,1419,120,210,1299
12621,26928,الفرع العلمي,2023-2022,حمص,يارا محمد عاصي,رشا,دراسة حرة,ناجح,1822,227,162,1660
12622,26929,الفرع العلمي,2023-2022,حمص,يارا محمد غريب,حفيظه,دراسة حرة,ناجح,1561,227,179,1382
12624,26933,الفرع العلمي,2023-2022,حمص,ياسمين علي برنيك,فتحيه,دراسة حرة,ناجح,1477,140,141,1337
